In [1]:
import pandas as pd # Make sure that you have openpyxl installed so that you can open newer excel files
import numpy as np
import os 
from zipfile import is_zipfile, ZipFile
from datetime import datetime, date
from plotnine import ggplot, geom_line, aes, theme_minimal
from plotnine3d import ggplot_3d, geom_polygon_3d
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [2]:
def extractFolder(inputFilesDir,unzippedFilesDir):
    for file in os.listdir(inputFilesDir):   # get the list of files
        if is_zipfile(inputFilesDir+file): # if it is a zipfile, extract it
            with ZipFile(inputFilesDir+file) as item: # treat the file as a zip
                item.extractall(unzippedFilesDir)  # extract it into a new folder

def cleanTermStructureSheet(filename,foldername,sheetName):
    df = pd.read_excel(foldername+filename, sheet_name=sheetName, header=1, engine='openpyxl')
    df.drop(['Unnamed: 0'], axis=1, inplace=True) # Blank column - artifact of the dataset
    df = df.iloc[8:,:] # Dropping all unnessary columns 
    df.rename(columns={"Main menu": "Tenor"}, inplace=True)
    df.set_index("Tenor", inplace=True)
    return df

def getDateFromFileName(filename):
    dateString = filename.split('_')[2]
    yearString = dateString[:4]
    monthString = dateString[4:6]
    dayString = dateString[6:]
    return date(int(yearString),int(monthString),int(dayString))

def getCountrieslist(inputFilesDir):
    return pd.read_csv(inputFilesDir+"countries.csv").columns.to_list()

def getFilteredFileList(unzippedFilesDir,fileType,sheetName):
    allFiles = os.listdir(unzippedFilesDir)
    filteredList = [file for file in allFiles if fileType in file]
    return filteredList 

def plotCurvesOverTime(coutriesDict, country, colourScale):
    countryData = pd.DataFrame(countriesDict[country]).sort_index(axis = 1)
    dates = countryData.columns
    tenors = countryData.index.values
    fig = go.Figure(data=[go.Surface(z=countryData.values, 
                      x=dates, 
                      y=tenors,
                      opacity = 0.8,
                      colorscale=colourScale
                      )])
    fig.update_layout(scene=dict(yaxis_title='Tenor',
                            xaxis_title='Date',
                            zaxis_title='Yield',
                            aspectratio = dict(x = 1, y = 1.4, z= 0.75)),
                    title=f'{country} Curve progression since 2022', autosize=False,
                    width=1000, height=700,
                    margin=dict(l=50, r=20, b=10, t=50), 
                    paper_bgcolor="LightSteelBlue"
                    )

    fig.show()

inputFilesDir = "inputData/"
unzippedFilesDir = "unzippedFiles/"
fileType = "Term_Structures"
sheetName = "RFR_spot_no_VA"

extractFolder(inputFilesDir,unzippedFilesDir)
countries = getCountrieslist(inputFilesDir)

countriesDict = {}
for country in countries:
    countriesDict[country] = {}

for filename in getFilteredFileList(unzippedFilesDir,fileType,sheetName):
    dateIndex = getDateFromFileName(filename)
    allTermStructures = cleanTermStructureSheet(filename,unzippedFilesDir,sheetName)
    for country in countries:
        countriesDict[country][dateIndex] =  allTermStructures.loc[:,country]


In [3]:
plotCurvesOverTime(countriesDict,"United States", "Reds")

In [4]:
plotCurvesOverTime(countriesDict,"Euro", "Blues")